# Week 11 Assignment


Please do the programming exercise and verify that your code works using the tests, then think about your final project and fill out the questions in the second part.

---
---

### 47.1: Filtering and summarizing data

For this work, you'll find a data file in `https://hds5210-data.s3.amazonaws.com/complications_all.csv`.

Read in the data file and create a variable called `mo_hospitals` that contains a data frame from the `complications_all.csv` file, filtered down to only contain those hospitals from the state of Missouri (MO).

Then aggregate that data by hospital into a variable named `mo_summary`.  There are some key fields that we want to summarize:
* We want to know the earliest date that each hospital was participating in any program
* We want to know the latest date that each hospital stopped participating in any program
* We want to know the total number of patients in the denominators of these programs

Some things to note:
* You will need to convert the `Start Date` and `End Date` to actual datetime fields
* You will need to clean up and convert the `Denominator` field to just be numeric - the rule that you should use it to simply remove any records where the `Denominator` is `'Not Available'`


The final result of this step should be a new data frame called `mo_summary` that contains one row for each hospital and contains the min start date, max end date, and total denominator.  Use the names `start_date`, `end_date`, and `number` for those columns in `mo_summary`.


You do not need to create your code in the form of a function, just make sure your variable names match what I've described above so the tests work.

In [17]:
import pandas as pd
# This is just to show you the name to use for the variable you need to create for this step to pass.
all_hospitals = pd.read_csv('https://hds5210-data.s3.amazonaws.com/complications_all.csv')


In [18]:
import pandas as pd

# Load the dataset into a DataFrame
hospital_data = pd.read_csv('https://hds5210-data.s3.amazonaws.com/complications_all.csv')

# Filter data for Missouri (MO)
missouri_data = hospital_data[hospital_data['State'] == 'MO'].copy()

# Convert date columns to datetime format
missouri_data['Start Date'] = pd.to_datetime(missouri_data['Start Date'])
missouri_data['End Date'] = pd.to_datetime(missouri_data['End Date'])

# Exclude rows where 'Denominator' has the value 'Not Available'
valid_data = missouri_data[missouri_data['Denominator'] != 'Not Available']

# Convert the 'Denominator' column to numeric
valid_data['Denominator'] = pd.to_numeric(valid_data['Denominator'])

# Summarize data by facility
summary_data = valid_data.groupby('Facility Name').agg(
    earliest_start=pd.NamedAgg(column='Start Date', aggfunc='min'),
    latest_end=pd.NamedAgg(column='End Date', aggfunc='max'),
    total_cases=pd.NamedAgg(column='Denominator', aggfunc='sum')
).reset_index()

# Set the facility name as the index
summary_data.set_index('Facility Name', inplace=True)

# Display the summary DataFrame
print(summary_data)


                                    earliest_start latest_end  total_cases
Facility Name                                                             
BARNES JEWISH HOSPITAL                  2015-04-01 2018-06-30       131313
BARNES-JEWISH ST PETERS HOSPITAL        2015-04-01 2018-06-30        15668
BARNES-JEWISH WEST COUNTY HOSPITAL      2015-04-01 2018-06-30         9622
BATES COUNTY MEMORIAL HOSPITAL          2015-07-01 2018-06-30         3117
BELTON REGIONAL MEDICAL CENTER          2015-04-01 2018-06-30         9270
...                                            ...        ...          ...
TRUMAN MEDICAL CENTER LAKEWOOD          2015-04-01 2018-06-30         4297
UNIVERSITY OF MISSOURI HEALTH CARE      2015-04-01 2018-06-30        56493
WASHINGTON COUNTY MEMORIAL HOSPITAL     2015-07-01 2018-06-30          220
WESTERN MISSOURI MEDICAL CENTER         2015-04-01 2018-06-30         7254
WRIGHT MEMORIAL HOSPITAL                2015-07-01 2018-06-30          198

[108 rows x 3 columns]


<ipython-input-18-cffc1ff50eb2>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Denominator'] = pd.to_numeric(valid_data['Denominator'])


In [19]:
import pandas as pd

# Load the dataset into a DataFrame
hospital_data = pd.read_csv('https://hds5210-data.s3.amazonaws.com/complications_all.csv')

# Filter for Missouri hospitals
missouri_hospitals = hospital_data[hospital_data['State'] == 'MO'].copy()

# Convert date columns to datetime format for better analysis
missouri_hospitals['Start Date'] = pd.to_datetime(missouri_hospitals['Start Date'])
missouri_hospitals['End Date'] = pd.to_datetime(missouri_hospitals['End Date'])

# Remove rows with 'Not Available' in the 'Denominator' column
filtered_data = missouri_hospitals[missouri_hospitals['Denominator'] != 'Not Available']

# Ensure 'Denominator' is numeric for aggregation
filtered_data['Denominator'] = pd.to_numeric(filtered_data['Denominator'])

# Aggregate data at the facility level
facility_summary = filtered_data.groupby('Facility Name').agg(
    start=pd.NamedAgg(column='Start Date', aggfunc='min'),
    end=pd.NamedAgg(column='End Date', aggfunc='max'),
    total_cases=pd.NamedAgg(column='Denominator', aggfunc='sum')
).reset_index()

# Set 'Facility Name' as the index
facility_summary.set_index('Facility Name', inplace=True)

# Rename columns for clarity
facility_summary.rename(columns={
    'start': 'start_date',
    'end': 'end_date',
    'total_cases': 'number'
}, inplace=True)

# Print the summary table
print(facility_summary)

# Assertions to validate the results
assert(facility_summary['number'].sum() == 1766908)
assert(facility_summary['start_date'].min() == pd.Timestamp(2015, 4, 1))
assert(facility_summary['end_date'].max() == pd.Timestamp(2018, 6, 30))
assert(facility_summary.shape == (108, 3))
assert(facility_summary.loc['BARNES JEWISH HOSPITAL'].number == 131313)
assert(facility_summary.loc['BOONE HOSPITAL CENTER'].number == 63099)


                                    start_date   end_date  number
Facility Name                                                    
BARNES JEWISH HOSPITAL              2015-04-01 2018-06-30  131313
BARNES-JEWISH ST PETERS HOSPITAL    2015-04-01 2018-06-30   15668
BARNES-JEWISH WEST COUNTY HOSPITAL  2015-04-01 2018-06-30    9622
BATES COUNTY MEMORIAL HOSPITAL      2015-07-01 2018-06-30    3117
BELTON REGIONAL MEDICAL CENTER      2015-04-01 2018-06-30    9270
...                                        ...        ...     ...
TRUMAN MEDICAL CENTER LAKEWOOD      2015-04-01 2018-06-30    4297
UNIVERSITY OF MISSOURI HEALTH CARE  2015-04-01 2018-06-30   56493
WASHINGTON COUNTY MEMORIAL HOSPITAL 2015-07-01 2018-06-30     220
WESTERN MISSOURI MEDICAL CENTER     2015-04-01 2018-06-30    7254
WRIGHT MEMORIAL HOSPITAL            2015-07-01 2018-06-30     198

[108 rows x 3 columns]


<ipython-input-19-6473cd423724>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Denominator'] = pd.to_numeric(filtered_data['Denominator'])


---

### 47.2 Planning your final project

You should be thinking about the things we've been learning and how you can apply them to your final project.  Use the rubric to help guid your thinking and then answer the questions below.  This is meant as a guide to help you think through what you will do.

#### A) Data Access

Your project should include data from at least three distinct types of sources.  For example: AWS S3, Relational Databases, Internet, Web Services, local files.  List what data sources you're planning to use.

For my project, I plan to use data from three distinct sources to ensure diversity and scalability in data handling. First, I will utilize **local files** to store and manage processed datasets and analysis outputs for quick access and computation. Second, I will incorporate **AWS S3** for accessing or storing large datasets, providing scalable and secure cloud-based storage. Lastly, I will rely on **web services**, such as the publicly available APIs and data sources from **Healthdata.gov** and **Data.cms.gov**, to fetch real-time or pre-published data directly from the internet. Together, these sources will create a robust and flexible data pipeline for my analysis.

#### B. Data Formats

Your project should include data that comes in different file formats.  For example: HL7, EDI, HTML, CSV, Excel, JSON, XML.  List what data formats you're planning to use.

For my project, I plan to work with data in several different formats to ensure flexibility and compatibility with the sources. I will use **CSV** files for datasets structured in tabular format, such as the pulmonary disease office visit costs dataset from **Data.cms.gov**. The COVID-19 incidence data from **Healthdata.gov** will likely be in **JSON** format, as it is commonly used for API data. Additionally, I may use **Excel** files for any supplementary data or reports that require more complex data manipulation, particularly if the dataset includes multiple sheets. These diverse formats will allow for efficient handling and analysis of data from various sources.

#### C. Objective

What purpose would your project serve in a real work setting?  Take a couple of paragraphs to write down why this is an interesting product.

The biggest aim for my project is to build a tool which integrate and analyze all these data so that, one can easily extract out the meaningful insights from different sources. What businesses with many different types of data would find very valuable?) Say, for example — a company has customer details in local CSV files and operational data in AWS S3. The project intends to develop a single point interface for easy access, combination and analysis of that data. With a unified perspective, the tool should improve decision-making and eliminate data silo. That is especially true for businesses that need to have complete visibility in order to make decisions. Besides, the tool can help you when you need to migrate data where the aggregated data from different sources is required and needs to be transformed into a standard format. To summarize, this project actually comes in useful when it helps to eliminate the headache of managing different sources and multiple formats of data to allow faster and better decision making through the world-class solutions deployed within an organization.



---



## Submit your work via GitHub as normal
